# Runner Reidentification

in this notebook we start combining the track ids from different videos by computing similarity between the feature of each, runner (distinguished by trackid), after that we(I)(python)(M2 chip) use hungarian algorithm to assign a runner to another

So the steps are:

- select frontal cameras
- aggregate feature by trackid
- compute similarity matrix
- assignment

## Dependencies

we firs import all the dependencies needed:

- pandas - to manipulate dataframe
- numpy - for matrix operations
- select_bib_number - custom function that select ( given feature of all relevation for one track id ) the best bib number of the runner
- aggregate_features - aggregate ( given feature of all relevation for one track id ) all the feature of a runner
- temporal_overlap - checks if
- compute_similarity
- build_similarity_matrix

In [4]:
import pandas as pd
import numpy as np

from utils import aggregate_features, build_similarity_matrix

In [5]:
#useful variables

dataset_path = 'set_videos_2/info_runners_DATASET.csv'
RightVideo_LeftVideo_features_output_path = 'set_videos_2/results_static_similarity_metric/RightVideo_LeftVideo_only_features.csv'
RightVideo_agg_output_path = 'set_videos_2/results_static_similarity_metric/RightVideo_thresholded_detections_aggregated_features.csv'
LeftVideo_agg_output_path = 'set_videos_2/results_static_similarity_metric/LeftVideo_thresholded_detections_aggregated_features.csv'

## STEP 1 - Select frontal cameras

In [6]:
df = pd.read_csv(dataset_path)

#to reduce the error on track ids we select only the runners near to the camera with box_height threshold
thresolded_runners = df[df['y2r'] - df['y1r'] > 550]

In [9]:
#select the two frontal camera
df_RightVideo = thresolded_runners[thresolded_runners['cam'] == 'FD']
df_LeftVideo = thresolded_runners[thresolded_runners['cam'] == 'FI']

In [10]:
print(f"Right frontal camera runner detected: {len(df_RightVideo['trackid'].value_counts())}/? real runner")
print(f"Left frontal camera runner detected: {len(df_LeftVideo['trackid'].value_counts())}/? real runner")

Right frontal camera runner detected: 6/? real runner
Left frontal camera runner detected: 6/? real runner


## Step 2 - Aggregate feature by trackid

In [11]:
track_ids_RightVideo = df_RightVideo.groupby("trackid")  # group by track id
track_ids_LeftVideo = df_LeftVideo.groupby("trackid")  # group by track id

#Right camera aggregation
tracks_list_RightVideo = []
for track_id, group in track_ids_RightVideo:
    feature_vector = aggregate_features(group)
    tracks_list_RightVideo.append(feature_vector)

#Left camera aggregation
tracks_list_LeftVideo = []
for track_id, group in track_ids_LeftVideo:
    feature_vector = aggregate_features(group)
    tracks_list_LeftVideo.append(feature_vector)

#now we have two list containing for each runner(trackid) his aggregated features

## Step 3 - Compute similarity matrix

In [12]:
sim_matrix = build_similarity_matrix(tracks_list_RightVideo, tracks_list_LeftVideo, dynamic_metric=True)

## Step 4 - Assignment

In [13]:
from scipy.optimize import linear_sum_assignment

runner_RightVideo, runner_LeftVideo = linear_sum_assignment(sim_matrix) # Hungarian algorithm implementazion
for track in tracks_list_RightVideo:
    track['match_id'] = None

for track in tracks_list_LeftVideo:
    track['match_id'] = None

match_id = 0
for c, cons in zip(runner_RightVideo, runner_LeftVideo):
    match_id +=1
    tracks_list_RightVideo[c]['match_id'] = match_id
    tracks_list_LeftVideo[cons]['match_id'] = match_id

## Aggregate features by match id


In [14]:
# convert list series into Pandas Dataframe
df_tracks_RightVideo = pd.DataFrame(tracks_list_RightVideo)
df_tracks_LeftVideo = pd.DataFrame(tracks_list_LeftVideo)

#concat
df_all = pd.concat([df_tracks_RightVideo, df_tracks_LeftVideo], ignore_index=True, sort=False)
grouped = df_all.groupby("match_id")  # group by match id

#Right camera aggregation
match_id_list = []
for match_id, group in grouped:
    feature_vector = aggregate_features(group, with_frame=False)
    match_id_list.append(feature_vector)
#now we have two list containing for each runner(trackid) his aggregated features

In [15]:
runners_RightVideo_LeftVideo_features = pd.DataFrame(match_id_list)
runners_RightVideo_LeftVideo_features.to_csv(RightVideo_LeftVideo_features_output_path, index=False)

## Save

Add match id to relevation dataframe

In [11]:
df_RightVideo = df_RightVideo.merge(df_tracks_RightVideo[['trackid', 'match_id']], on='trackid', how='left')
df_LeftVideo = df_LeftVideo.merge(df_tracks_LeftVideo[['trackid', 'match_id']], on='trackid', how='left')


Once we have the match_id we can edit the multiple features per runner with the aggregate one, by joining the two dataframes

In [12]:
#keep only the non trait features
df_RightVideo_no_trait = df_RightVideo[["match_id", "cam","filename","nframe","trackid","x1r","y1r","x2r","y2r","x1b","y1b","x2b","y2b"]]
#aggregate with the aggregate features dataframe, on match_id
df_RightVideo_agg = df_RightVideo_no_trait.merge(runners_RightVideo_LeftVideo_features, on='match_id', how='left')

#keep only the non trait features
df_LeftVideo_no_trait = df_LeftVideo[["match_id", "cam","filename","nframe","trackid","x1r","y1r","x2r","y2r","x1b","y1b","x2b","y2b"]]
#aggregate with the aggregate features dataframe, on match_id
df_LeftVideo_agg = df_LeftVideo_no_trait.merge(runners_RightVideo_LeftVideo_features, on='match_id', how='left')

df_RightVideo_agg.to_csv(RightVideo_agg_output_path, index=False)
df_LeftVideo_agg.to_csv(LeftVideo_agg_output_path, index=False)
